In [1]:
from collections import Counter, defaultdict

In [2]:
from plot_utils import *

In [3]:
from retention_utils import *

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


<IPython.core.display.Javascript object>

R[write to console]: Error in withVisible({ : 找不到对象'plot_attrition_df'




Error in withVisible({ : 找不到对象'plot_attrition_df'


In [4]:
import jsonmemo as jsonmemo_module
jsonmemo_module.set_lowmem(True)

In [5]:


def get_condition_to_installs_for_random_assignment_abtest():
  abtest_name = 'difficulty_selection_screen_and_choose_difficulty_frequency'
  groups = ['survey', 'nodefault_forcedchoice_userchoice', 'survey_nochoice_nothing', 'survey_nochoice_easy', 'survey_nochoice_medium', 'survey_nochoice_hard']
  #condition_to_installs = get_conditions_to_install_list_in_abtest_unstrict(abtest_name)
  condition_to_installs = get_conditions_to_install_list_in_abtest(abtest_name, groups)
  for k in list(condition_to_installs.keys()):
    if k not in groups:
      del condition_to_installs[k]
  return condition_to_installs


In [6]:

#for install in installs_with_experiment_vars:
@msgpackmemo1arg
def get_domain_to_intervention_including_empty_to_session_lengths_for_install(install):
  seconds_on_domain_per_session = get_collection_for_install(install, 'synced:seconds_on_domain_per_session')
  interventions_active_for_domain_and_session = get_collection_for_install(install, 'synced:interventions_active_for_domain_and_session')
  domain_to_session_to_intervention = {}
  for x in interventions_active_for_domain_and_session:
    if x.get('val') is None:
      continue
    interventions_active = json.loads(x['val'])
    if len(interventions_active) == 0:
      intervention_name = 'no_intervention'
      #continue
    else:
      intervention_name = interventions_active[0]
    domain = x['key']
    session_id = x['key2']
    if domain not in domain_to_session_to_intervention:
      domain_to_session_to_intervention[domain] = {}
    domain_to_session_to_intervention[domain][session_id] = intervention_name
  
  domain_to_intervention_to_session_lengths = {}
  for x in seconds_on_domain_per_session:
    if 'key' not in x:
      print('missing key in seconds_on_domain_per_session')
      print(x)
      continue
    domain = x['key']
    session_id = x['key2']
    time_spent = x['val']
    if domain not in domain_to_session_to_intervention:
      continue
    intervention_name = domain_to_session_to_intervention[domain].get(session_id)
    if intervention_name is None:
      continue
    if domain not in domain_to_intervention_to_session_lengths:
      domain_to_intervention_to_session_lengths[domain] = {}
    if intervention_name not in domain_to_intervention_to_session_lengths[domain]:
      domain_to_intervention_to_session_lengths[domain][intervention_name] = []
    domain_to_intervention_to_session_lengths[domain][intervention_name].append(time_spent)
  return domain_to_intervention_to_session_lengths
  

In [7]:

#for install in installs_with_experiment_vars:
@msgpackmemo1arg
def get_domain_to_intervention_to_session_lengths_for_install(install):
  seconds_on_domain_per_session = get_collection_for_install(install, 'synced:seconds_on_domain_per_session')
  interventions_active_for_domain_and_session = get_collection_for_install(install, 'synced:interventions_active_for_domain_and_session')
  
  domain_to_session_to_intervention = {}
  for x in interventions_active_for_domain_and_session:
    if x.get('val') is None:
      continue
    interventions_active = json.loads(x['val'])
    if len(interventions_active) == 0:
      continue
    intervention_name = interventions_active[0]
    domain = x['key']
    session_id = x['key2']
    if domain not in domain_to_session_to_intervention:
      domain_to_session_to_intervention[domain] = {}
    domain_to_session_to_intervention[domain][session_id] = intervention_name
  
  domain_to_intervention_to_session_lengths = {}
  for x in seconds_on_domain_per_session:
    if 'key' not in x:
      print('missing key in seconds_on_domain_per_session')
      print(x)
      continue
    domain = x['key']
    session_id = x['key2']
    time_spent = x['val']
    if domain not in domain_to_session_to_intervention:
      continue
    intervention_name = domain_to_session_to_intervention[domain].get(session_id)
    if intervention_name is None:
      continue
    if domain not in domain_to_intervention_to_session_lengths:
      domain_to_intervention_to_session_lengths[domain] = {}
    if intervention_name not in domain_to_intervention_to_session_lengths[domain]:
      domain_to_intervention_to_session_lengths[domain][intervention_name] = []
    domain_to_intervention_to_session_lengths[domain][intervention_name].append(time_spent)
  return domain_to_intervention_to_session_lengths
  

In [8]:
# for condition,install_list in get_condition_to_installs_for_random_assignment_abtest().items():
#   for install_id in install_list:
#     try:
#       domain_to_intervention_to_session_lengths = get_domain_to_intervention_to_session_lengths_for_install(install_id)
#     except:
#       continue

In [9]:
@memoize
def get_install_id_to_domain_to_intervention_including_empty():
  output = {}
  for install_id in get_install_to_dates_active_list().keys():
    try:
      domain_to_intervention_to_session_lengths = get_domain_to_intervention_including_empty_to_session_lengths_for_install(install_id)
      output[install_id] = domain_to_intervention_to_session_lengths
    except:
      continue
  return output

In [10]:
@memoize
def get_install_id_to_domain_to_intervention():
  output = {}
  for install_id in get_install_to_dates_active_list().keys():
    try:
      domain_to_intervention_to_session_lengths = get_domain_to_intervention_to_session_lengths_for_install(install_id)
      output[install_id] = domain_to_intervention_to_session_lengths
    except:
      continue
  return output

In [11]:
from browser_libs import *

install_id_to_user = get_install_id_to_user()
user_set = set()
for install_id,domain_to_intervention_to_session_lengths in get_install_id_to_domain_to_intervention_including_empty().items():
  if 'www.facebook.com' not in domain_to_intervention_to_session_lengths:
    continue
  is_valid_user = False
  for intervention,session_lengths in domain_to_intervention_to_session_lengths['www.facebook.com'].items():
    if (not intervention.startswith('facebook/')) and (intervention != 'no_intervention'):
      continue
    if len(session_lengths) > 0:
      is_valid_user = True
  if is_valid_user:
    if install_id in install_id_to_user:
      user_id = install_id_to_user[install_id]
      user_set.add(user_id)


exception in msgpackmemo1arg for file 2020_09_10/get_domain_to_intervention_including_empty_to_session_lengths_for_install/96c2e7a861aa704d3e532ecc.msgpack
[Errno 2] No such file or directory: '2020_09_10/get_domain_to_intervention_including_empty_to_session_lengths_for_install/96c2e7a861aa704d3e532ecc.msgpack'
performing computation 2020_09_10/get_domain_to_intervention_including_empty_to_session_lengths_for_install/96c2e7a861aa704d3e532ecc.msgpack for arg 96c2e7a861aa704d3e532ecc
exception in msgpackmemo1arg for file 2020_09_10/get_domain_to_intervention_including_empty_to_session_lengths_for_install/4167238e18e1a24e7d0e98b8.msgpack
[Errno 2] No such file or directory: '2020_09_10/get_domain_to_intervention_including_empty_to_session_lengths_for_install/4167238e18e1a24e7d0e98b8.msgpack'
performing computation 2020_09_10/get_domain_to_intervention_including_empty_to_session_lengths_for_install/4167238e18e1a24e7d0e98b8.msgpack for arg 4167238e18e1a24e7d0e98b8
exception in msgpackmemo1a

In [12]:
len(user_set)

15664

In [13]:
intervention_to_num_samples = Counter()
for install_id,domain_to_intervention_to_session_lengths in get_install_id_to_domain_to_intervention_including_empty().items():
  if 'www.facebook.com' not in domain_to_intervention_to_session_lengths:
    continue
  for intervention,session_lengths in domain_to_intervention_to_session_lengths['www.facebook.com'].items():
    if (not intervention.startswith('facebook/')) and (intervention != 'no_intervention'):
      continue
    intervention_to_num_samples[intervention] += len(session_lengths)


In [14]:
#intervention_to_session_length_list['no_intervention']

In [15]:
intervention_to_session_length_list = defaultdict(list)
for install_id,domain_to_intervention_to_session_lengths in get_install_id_to_domain_to_intervention_including_empty().items():
  #continue
  if 'www.facebook.com' not in domain_to_intervention_to_session_lengths:
    continue
  for intervention,session_lengths in domain_to_intervention_to_session_lengths['www.facebook.com'].items():
    if (not intervention.startswith('facebook/')) and (intervention != 'no_intervention'):
      continue
    if intervention_to_num_samples[intervention] < 1000:
      continue
    for session_length in session_lengths:
      intervention_to_session_length_list[intervention].append(session_length)

In [16]:
num_samples = sum([len(session_lengths) for intervention,session_lengths in intervention_to_session_length_list.items()])
print(num_samples)

15926160


In [17]:
intervention_to_median_session_length = {intervention: np.median(session_lengths) for intervention,session_lengths in intervention_to_session_length_list.items()}

In [18]:
intervention_difficulties = {
  'no_intervention': 0,
  'facebook/feed_injection_timer': 1, # 1 1 1
  'facebook/remove_clickbait': 1, # 1 1 1
  'facebook/rich_notifications': 1, # 1 2 1
  'facebook/show_timer_banner': 1, # 1 1 3
  'facebook/toast_notifications': 2, # 1 2 3
  'facebook/remove_comments': 2, # 2 1 3
  'facebook/remove_news_feed': 2, # 2 2 2
  'facebook/make_user_wait': 2, # 3 2 2
  'facebook/block_after_interval_per_visit': 2, # 3 2 2
  'facebook/scroll_blocker': 3, # 3 3 3
  'facebook/show_user_info_interstitial': 3, # 3 3 3
  'facebook/close_tab_timer': 3, # 3 3 3
}

In [19]:
intervention_names = {
  'no_intervention': 'No Intervention',
  'facebook/feed_injection_timer': 'Inject Feed Timer', # 1 1 1
  'facebook/remove_clickbait': 'Remove Clickbait', # 1 1 1
  'facebook/rich_notifications': 'Rich Notifications', # 1 2 1
  'facebook/show_timer_banner': 'Timer Banner', # 1 1 3
  'facebook/toast_notifications': 'Toast Notifications', # 1 2 3
  'facebook/remove_comments': 'Remove Comments', # 2 1 3
  'facebook/remove_news_feed': 'Remove News Feed', # 2 2 2
  'facebook/make_user_wait': 'Make User Wait', # 3 2 2
  'facebook/block_after_interval_per_visit': 'Block After Interval', # 3 2 2
  'facebook/scroll_blocker': 'Scroll Blocker', # 3 3 3
  'facebook/show_user_info_interstitial': 'Info Interstitial', # 3 3 3
  'facebook/close_tab_timer': 'Close Tab Timer', # 3 3 3
}

In [20]:
intervention_to_median_session_length = {intervention_names[intervention]: np.median(session_lengths) for intervention,session_lengths in intervention_to_session_length_list.items() if intervention in intervention_names}

In [21]:
plot_dict_as_bar(intervention_to_median_session_length, title='Time spent on Facebook in the presence of interventions', ylabel='Median Facebook session length, in seconds', font=dict(size=13))

In [22]:
plot_dict_as_bar({(str(intervention_difficulties.get(intervention, 0)) + intervention): median_session_length for intervention,median_session_length in intervention_to_median_session_length.items()})

In [23]:
difficulty_to_session_length_list = defaultdict(list)
for install_id,domain_to_intervention_to_session_lengths in get_install_id_to_domain_to_intervention_including_empty().items():
  if 'www.facebook.com' not in domain_to_intervention_to_session_lengths:
    continue
  for intervention,session_lengths in domain_to_intervention_to_session_lengths['www.facebook.com'].items():
    if (not intervention.startswith('facebook/')) and (intervention != 'no_intervention'):
      continue
    if intervention_to_num_samples[intervention] < 1000:
      continue
    if intervention not in intervention_difficulties:
      continue
    difficulty = intervention_difficulties[intervention]
    for session_length in session_lengths:
      difficulty_to_session_length_list[difficulty].append(session_length)

In [24]:
difficulty_to_median_session_length = {difficulty: np.median(session_lengths) for difficulty,session_lengths in difficulty_to_session_length_list.items()}

In [25]:
import math

In [26]:
difficulty_to_mean_session_length = {difficulty: np.mean([math.log(x) for x in session_lengths]) for difficulty,session_lengths in difficulty_to_session_length_list.items()}

In [27]:
import scipy
import scipy.stats

scipy.stats.ttest_ind([math.log(x) for x in difficulty_to_session_length_list[1]], [math.log(x) for x in difficulty_to_session_length_list[2]])

Ttest_indResult(statistic=100.22055153801293, pvalue=0.0)

In [28]:
scipy.stats.mannwhitneyu([math.log(x) for x in difficulty_to_session_length_list[2]], [math.log(x) for x in difficulty_to_session_length_list[3]])

MannwhitneyuResult(statistic=2786088138253.0, pvalue=0.0)

In [29]:
x=scipy.stats.mannwhitneyu([x for x in difficulty_to_session_length_list[0]], [x for x in difficulty_to_session_length_list[1]])

In [30]:
scipy.stats.mannwhitneyu([x for x in difficulty_to_session_length_list[2]], [x for x in difficulty_to_session_length_list[3]])

MannwhitneyuResult(statistic=2786088138253.0, pvalue=0.0)

In [31]:
scipy.stats.ttest_ind([math.log(x) for x in difficulty_to_session_length_list[2]], [math.log(x) for x in difficulty_to_session_length_list[3]])

Ttest_indResult(statistic=87.77022932389526, pvalue=0.0)

In [32]:
plot_dict_as_bar(difficulty_to_median_session_length, remap_labels={0: 'No Intervention', 1: 'Easy', 2: 'Medium', 3: 'Hard'}, title='Intervention difficulty vs time spent on Facebook', xlabel='Difficulty level of intervention shown', ylabel='Median session duration (seconds)', orientation='v', font=dict(size=22))

In [33]:
plot_dict_as_bar(intervention_to_num_samples)

In [34]:
domain_to_num_samples = Counter()
for install_id,domain_to_intervention_to_session_lengths in get_install_id_to_domain_to_intervention().items():
  for domain,intervention_to_session_lengths in domain_to_intervention_to_session_lengths.items():
    for intervention,session_lengths in intervention_to_session_lengths.items():
      domain_to_num_samples[domain] += len(session_lengths)

exception in msgpackmemo1arg for file 2020_09_10/get_domain_to_intervention_to_session_lengths_for_install/96c2e7a861aa704d3e532ecc.msgpack
[Errno 2] No such file or directory: '2020_09_10/get_domain_to_intervention_to_session_lengths_for_install/96c2e7a861aa704d3e532ecc.msgpack'
performing computation 2020_09_10/get_domain_to_intervention_to_session_lengths_for_install/96c2e7a861aa704d3e532ecc.msgpack for arg 96c2e7a861aa704d3e532ecc
exception in msgpackmemo1arg for file 2020_09_10/get_domain_to_intervention_to_session_lengths_for_install/4167238e18e1a24e7d0e98b8.msgpack
[Errno 2] No such file or directory: '2020_09_10/get_domain_to_intervention_to_session_lengths_for_install/4167238e18e1a24e7d0e98b8.msgpack'
performing computation 2020_09_10/get_domain_to_intervention_to_session_lengths_for_install/4167238e18e1a24e7d0e98b8.msgpack for arg 4167238e18e1a24e7d0e98b8
exception in msgpackmemo1arg for file 2020_09_10/get_domain_to_intervention_to_session_lengths_for_install/414f544cde475af

In [35]:
plot_dict_as_bar(domain_to_num_samples)

In [36]:
# domain_to_intervention_to_session_lengths = get_domain_to_intervention_to_session_lengths_for_install()